<a href="https://colab.research.google.com/github/vishnu-chand/planetGenerator/blob/main/planetGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### init

In [ ]:
%load_ext autoreload
%autoreload 2

import os
if '' or not os.path.exists('/content/pixUtils'):
    !git clone https://github.com/vishnu-chand/pixUtils.git;mv pixUtils deleteMe;mv deleteMe/pixUtils .;rm -rf deleteMe
    !pip install --upgrade --no-cache-dir gdown
    !git clone https://github.com/odegeasslbc/FastGAN-pytorch.git
    from pixUtils import *

import requests
from pixUtils import *
from urllib.parse import unquote


def decodeUrl(line):
    line = line.encode().decode('unicode-escape')
    url = unquote(line)
    while url != line:
        url, line = unquote(line), url
    name = basename(url)
    for sym in '? :'.split():
        name = name.split(sym)[0]
    return f"{dirname(url)}/{name}"


def googleSearch(htmlPath):
    htmlPath = [h for h in htmlPath.split('\n') if h.strip()][-1].strip()
    with open(htmlPath, 'r') as book:
        lines = book.read()
    imUrls = list()
    lines = lines.split('</script><div ng-non-bindable')[1]
    for line in lines.split('https://')[1:]:
        try:
            line, w, h = line.split(',')[:3]
            w, h = int(w), int(h.split(']')[0])
        except:
            continue
        url = decodeUrl(line[:-1])
        if 'encrypted-tbn0.gstatic.com' in url:
            continue
        imUrls.append([h, w, f"https://{url}"])
    return imUrls


def yandexSearch(htmlPath):
    from bs4 import BeautifulSoup as bs4
    htmlPath = [h for h in htmlPath.split('\n') if h.strip()][-1].strip()
    soup = bs4(open(htmlPath), 'html.parser')
    res = list()
    className = 'serp-item serp-item_type_search serp-item_group_search serp'
    for ix, img in enumerate(soup.find_all(attrs={'class': lambda e: e.startswith(className) if e else False})):
        data = json.loads(img.get('data-bem'))['serp-item']['preview'][0]
        res.append((data['h'], data['w'], decodeUrl(data['url'])))
    # res = list()
    # className = 'serp-item__link'
    # for ix, img in enumerate(soup.find_all(attrs={'class': lambda e: e.startswith(className) if e else False})):
    #     imgUrl = img['href'].split('img_url=')[1].split('&')[0]
    #     data = json.loads(img.get('data-bem'))['serp-item']['preview'][0]
    #     res.append((data['h'], data['w'], decodeUrl(data['url'])))
    #     raise NotImplemented
    return res


def downloadImg(filepath, imgUrl, wh, timeout, verbose):
    imShape = None
    try:
        headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_2) '
                                 'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36;'}
        r = requests.get(imgUrl, headers=headers, stream=True, timeout=timeout)
        with open(filepath, 'wb') as f_write:
            for chunk in r.iter_content(1024 * 1024):
                f_write.write(chunk)
        img = cv2.imread(filepath)
        if img is not None:
            imShape = '_'.join([str(i) for i in img.shape])
            if os.path.splitext(imgUrl)[1] != '.jpg':
                dirop(filepath, rm=' ')
                filepath = f"{filename(filepath, returnPath=True)}.jpg"
                cv2.imwrite(filepath, img)
        else:
            dirop(filepath, rm=' ')
            raise Exception("downloading failed")
    except Exception as exp:
        if verbose:
            print(f"""
    {exp}
    imgUrl   : {imgUrl}
    filepath : {filepath}
    imShape  : {imShape}
    """)
    return filepath, imgUrl, wh, imShape


def crawler(inferencePool, desPath, imUrls, minSize, timeout, verbose):
    desDir, desName = dirname(desPath), basename(desPath)
    dones = [int(filename(f)) for f in rglob(f"{desDir}/**/*.*") if filename(f) != 'url2img']
    print("len(imUrls)", len(imUrls))
    jobs = list()
    for h, w, imgUrl in sorted(imUrls, key=lambda x: x[0] * x[1]):
        ext = os.path.splitext(imgUrl)[1] or '.png'
        fname = shaIt(imgUrl)
        filepath = dirop(f'{desDir}/{desName}/{fname}{ext.lower()}')
        if fname not in dones:
            if w > minSize and h > minSize:
                if verbose:
                    print([w, h], fname, imgUrl)
                jobs.append(inferencePool.submit(downloadImg, filepath=filepath, imgUrl=imgUrl, wh=f'{w}_{h}', timeout=timeout, verbose=verbose))
    ok, nok = 0, 0
    with open(f'{desDir}/url2img.log', 'a') as book:
        for job in jobs:
            filepath, imgUrl, wh, imShape = job.result()
            if imShape:
                msg = f"{wh}, {imShape}, {imgUrl}, {filepath}\n"
                book.write(msg)
                ok += 1
            else:
                nok += 1
    return ok, nok

Cloning into 'pixUtils'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 196 (delta 22), reused 67 (delta 22), pack-reused 129
Receiving objects: 100% (196/196), 11.70 MiB | 14.64 MiB/s, done.
Resolving deltas: 100% (81/81), done.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=537cdc77ebb3e27342328271c11fc2530dec087382a3231be392f0fff485348e
  Stored in directory: /tmp/pip-ephem-wheel-cache-wv1xlqjh/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2
Cloning into 'FastGAN-pytorch'...
remote: Enumerating objects: 108, done.
remote: Counting ob

In [ ]:
trainCode = '''
import os
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader
from torchvision import transforms
from torchvision import utils as vutils

import argparse
import random
from tqdm import tqdm

from models import weights_init, Discriminator, Generator
from operation import copy_G_params, load_params, get_dir
from operation import ImageFolder, InfiniteSamplerWrapper
from diffaug import DiffAugment

policy = 'color,translation'
import lpips

use_gpu = False
if torch.cuda.is_available():
    use_gpu = True

percept = lpips.PerceptualLoss(model='net-lin', net='vgg', use_gpu=use_gpu)


# torch.backends.cudnn.benchmark = True


def crop_image_by_part(image, part):
    hw = image.shape[2] // 2
    if part == 0:
        return image[:, :, :hw, :hw]
    if part == 1:
        return image[:, :, :hw, hw:]
    if part == 2:
        return image[:, :, hw:, :hw]
    if part == 3:
        return image[:, :, hw:, hw:]


def train_d(net, data, label="real"):
    """Train function of discriminator"""
    if label == "real":
        part = random.randint(0, 3)
        pred, [rec_all, rec_small, rec_part] = net(data, label, part=part)
        err = F.relu(torch.rand_like(pred) * 0.2 + 0.8 - pred).mean() + \
              percept(rec_all, F.interpolate(data, rec_all.shape[2])).sum() + \
              percept(rec_small, F.interpolate(data, rec_small.shape[2])).sum() + \
              percept(rec_part, F.interpolate(crop_image_by_part(data, part), rec_part.shape[2])).sum()
        err.backward()
        return pred.mean().item(), rec_all, rec_small, rec_part
    else:
        pred = net(data, label)
        err = F.relu(torch.rand_like(pred) * 0.2 + 0.8 + pred).mean()
        err.backward()
        return pred.mean().item()


def get_dir(args):
    import shutil, json
    task_name = f'{args.saveDir}/{args.name}'
    saved_model_folder = os.path.join(task_name, 'models')
    saved_image_folder = os.path.join(task_name, 'images')

    os.makedirs(saved_model_folder, exist_ok=True)
    os.makedirs(saved_image_folder, exist_ok=True)

    for f in os.listdir('./'):
        if '.py' in f:
            shutil.copy(f, task_name + '/' + f)

    with open(os.path.join(saved_model_folder, '../args.txt'), 'w') as f:
        json.dump(args.__dict__, f, indent=2)

    return saved_model_folder, saved_image_folder


def train(args):
    data_root = args.path
    total_iterations = args.iter
    checkpoint = args.ckpt
    batch_size = args.batch_size
    im_size = args.im_size
    ndf = 64
    ngf = 64
    nz = 256
    nlr = 0.0001
    nbeta1 = 0.5
    use_cuda = use_gpu
    multi_gpu = True
    dataloader_workers = 8
    current_iteration = 0
    save_interval = 10
    saved_model_folder, saved_image_folder = get_dir(args)

    device = torch.device("cpu")
    if use_cuda:
        device = torch.device("cuda:0")

    transform_list = [
        transforms.Resize((int(im_size), int(im_size))),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ]
    trans = transforms.Compose(transform_list)

    if 'lmdb' in data_root:
        from operation import MultiResolutionDataset
        dataset = MultiResolutionDataset(data_root, trans, 1024)
    else:
        dataset = ImageFolder(root=data_root, transform=trans)

    dataloader = iter(DataLoader(dataset, batch_size=batch_size, shuffle=False,
                                 sampler=InfiniteSamplerWrapper(dataset), num_workers=dataloader_workers, pin_memory=True))
    """
    loader = MultiEpochsDataLoader(dataset, batch_size=batch_size, 
                               shuffle=True, num_workers=dataloader_workers, 
                               pin_memory=True)
    dataloader = CudaDataLoader(loader, 'cuda')
    """

    # from model_s import Generator, Discriminator
    netG = Generator(ngf=ngf, nz=nz, im_size=im_size)
    netG.apply(weights_init)

    netD = Discriminator(ndf=ndf, im_size=im_size)
    netD.apply(weights_init)

    netG.to(device)
    netD.to(device)

    optimizerG = optim.AdamW(netG.parameters(), lr=nlr, betas=(nbeta1, 0.999))
    optimizerD = optim.AdamW(netD.parameters(), lr=nlr, betas=(nbeta1, 0.999))

    avg_param_G = copy_G_params(netG)

    fixed_noise = torch.FloatTensor(8, nz).normal_(0, 1).to(device)

    if checkpoint != 'None':
        print(f"loding weights from {checkpoint}")
        ckpt = torch.load(checkpoint, map_location=device)

        netG = loadWeights(ckpt['g'], netG)
        netD = loadWeights(ckpt['d'], netD)
        avg_param_G = ckpt['g_ema']
        optimizerG.load_state_dict(ckpt['opt_g'])
        optimizerD.load_state_dict(ckpt['opt_d'])
        # current_iteration = int(checkpoint.split('_')[-1].split('.')[0])
        del ckpt

    if multi_gpu:
        netG = nn.DataParallel(netG.to(device))
        netD = nn.DataParallel(netD.to(device))

    for iteration in tqdm(range(current_iteration, total_iterations + 1)):
        real_image = next(dataloader)
        real_image = real_image.to(device)
        current_batch_size = real_image.size(0)
        noise = torch.Tensor(current_batch_size, nz).normal_(0, 1).to(device)

        fake_images = netG(noise)

        real_image = DiffAugment(real_image, policy=policy)
        fake_images = [DiffAugment(fake, policy=policy) for fake in fake_images]

        ## 2. train Discriminator
        netD.zero_grad()

        err_dr, rec_img_all, rec_img_small, rec_img_part = train_d(netD, real_image, label="real")
        train_d(netD, [fi.detach() for fi in fake_images], label="fake")
        optimizerD.step()

        ## 3. train Generator
        netG.zero_grad()
        pred_g = netD(fake_images, "fake")
        err_g = -pred_g.mean()

        err_g.backward()
        optimizerG.step()

        for p, avg_p in zip(netG.parameters(), avg_param_G):
            avg_p.mul_(0.999).add_(0.001 * p.data)

        if iteration % 100 == 0:
            print("GAN: loss d: %.5f    loss g: %.5f" % (err_dr, -err_g.item()))

        if iteration % (save_interval * 10) == 0:
            backup_para = copy_G_params(netG)
            load_params(netG, avg_param_G)
            with torch.no_grad():
                vutils.save_image(netG(fixed_noise)[0].add(1).mul(0.5), saved_image_folder + '/%d.jpg' % iteration, nrow=4)
                vutils.save_image(torch.cat([
                    F.interpolate(real_image, 128),
                    rec_img_all, rec_img_small,
                    rec_img_part]).add(1).mul(0.5), saved_image_folder + '/rec_%d.jpg' % iteration)
            load_params(netG, backup_para)

        if iteration % (save_interval * 50) == 0 or iteration == total_iterations:
            backup_para = copy_G_params(netG)
            load_params(netG, avg_param_G)
            # torch.save({'g': netG.state_dict(), 'd': netD.state_dict()}, saved_model_folder + '/%d.pth' % iteration)
            load_params(netG, backup_para)
            torch.save({'g'    : netG.state_dict(),
                        'd'    : netD.state_dict(),
                        'g_ema': avg_param_G,
                        'opt_g': optimizerG.state_dict(),
                        'opt_d': optimizerD.state_dict()}, saved_model_folder + '/all_%d.pth' % iteration)


def loadWeights(ckpt, netG):
    class Temp(nn.Module):
        def __init__(self, model):
            super().__init__()
            self.module = model
    try:
        temp = Temp(netG)
        temp.load_state_dict(ckpt)
        model = temp.module
    except:
        netG.load_state_dict(ckpt)
        model = netG
    return model


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='region gan')

    parser.add_argument('--path', type=str, default='../lmdbs/art_landscape_1k', help='path of resource dataset, should be a folder that has one or many sub image folders inside')
    parser.add_argument('--cuda', type=int, default=0, help='index of gpu to use')
    parser.add_argument('--name', type=str, default='test1', help='experiment name')
    parser.add_argument('--iter', type=int, default=50000, help='number of iterations')
    parser.add_argument('--start_iter', type=int, default=0, help='the iteration to start training')
    parser.add_argument('--batch_size', type=int, default=6, help='mini batch number of images')
    parser.add_argument('--im_size', type=int, default=1024, help='image resolution')
    parser.add_argument('--ckpt', type=str, default='None', help='checkpoint weight path if have one')
    parser.add_argument('--saveDir', type=str, default='None', help='checkpoint weight path if have one')

    args = parser.parse_args()
    print(args)

    train(args)
'''

evalCode = '''
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import utils as vutils

import os
import random
import argparse
from tqdm import tqdm

from models import Generator


def load_params(model, new_param):
    for p, new_p in zip(model.parameters(), new_param):
        p.data.copy_(new_p)


def resize(img):
    return F.interpolate(img, size=256)


def batch_generate(zs, netG, batch=8):
    g_images = []
    with torch.no_grad():
        for i in range(len(zs) // batch):
            g_images.append(netG(zs[i * batch:(i + 1) * batch]).cpu())
        if len(zs) % batch > 0:
            g_images.append(netG(zs[-(len(zs) % batch):]).cpu())
    return torch.cat(g_images)


def batch_save(images, folder_name):
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)
    for i, image in enumerate(images):
        vutils.save_image(image.add(1).mul(0.5), folder_name + '/%d.jpg' % i)


if __name__ == "__main__":
    parser = argparse.ArgumentParser(
            description='generate images'
    )
    parser.add_argument('--ckpt', type=str)
    parser.add_argument('--artifacts', type=str, default=".", help='path to artifacts.')
    parser.add_argument('--cuda', type=int, default=None, help='index of gpu to use')
    parser.add_argument('--start_iter', type=int, default=6)
    parser.add_argument('--end_iter', type=int, default=10)

    parser.add_argument('--dist', type=str, default='.')
    parser.add_argument('--size', type=int, default=256)
    parser.add_argument('--batch', default=16, type=int, help='batch size')
    parser.add_argument('--n_sample', type=int, default=2000)
    parser.add_argument('--big', action='store_true')
    parser.add_argument('--im_size', type=int, default=1024)
    parser.add_argument('--desDir', type=str, default='')
    parser.add_argument('--seed', type=int, default=None)
    parser.set_defaults(big=False)
    args = parser.parse_args()
    seed = args.seed or torch.random.seed()
    if seed is not None:
        torch.manual_seed(seed)

    noise_dim = 256
    device = torch.device('cpu')
    if args.cuda is not None:
        device = torch.device('cuda:%d' % (args.cuda))

    net_ig = Generator(ngf=64, nz=noise_dim, nc=3, im_size=args.im_size)  # , big=args.big )
    net_ig.to(device)
    if ' ':
        epoch = 0
    # for epoch in [10000 * i for i in range(args.start_iter, args.end_iter + 1)]:
    #     ckpt = f"{args.artifacts}/models/{epoch}.pth"
        ckpt = args.ckpt
        checkpoint = torch.load(ckpt, map_location=lambda a, b: a)
        # Remove prefix `module`.
        checkpoint['g'] = {k.replace('module.', ''): v for k, v in checkpoint['g'].items()}
        net_ig.load_state_dict(checkpoint['g'])
        # load_params(net_ig, checkpoint['g_ema'])

        # net_ig.eval()
        print('load checkpoint success, epoch %d' % epoch)

        net_ig.to(device)

        del checkpoint

        dist = f"{args.desDir}/eval_{epoch}"
        os.makedirs(dist, exist_ok=True)

        with torch.no_grad():
            for i in tqdm(range(args.n_sample // args.batch)):
                noise = torch.randn(args.batch, noise_dim).to(device)
                g_imgs = net_ig(noise)[0]
                g_imgs = F.interpolate(g_imgs, 512)
                for j, g_img in enumerate(g_imgs):
                    vutils.save_image(g_img.add(1).mul(0.5),
                                      os.path.join(dist, '%d.png' % (i * args.batch + j)))  # , normalize=True, range=(-1,1))

'''

##### upload HTML

In [ ]:
from google.colab import files
print("""
Google image search for exoplanets and upload it html
eg: https://www.google.com/search?q=exoplanets&tbm=isch right click save html and upload here
""")
a = files.upload()


Google image search for exoplanets and upload it html
eg: https://www.google.com/search?q=exoplanets&tbm=isch right click save html and upload here



Saving exoplanets wallpaper - Google Search.html to exoplanets wallpaper - Google Search.html


##### step1: Build a spider to crawl into World Wide Web and collect images of exoplanets

In [ ]:
from concurrent.futures import ThreadPoolExecutor
inferencePool = ThreadPoolExecutor(max_workers=4)

def step1_crawler(inferencePool, htmlPaths, desPath):
    for htmlPath in htmlPaths:
        if 'google' in htmlPath.lower():
            imUrls = googleSearch(htmlPath)
        else:
            imUrls = yandexSearch(htmlPath)
        ok, nok = crawler(inferencePool, desPath, imUrls, minSize=384, timeout=10, verbose=False)
        print(f"{htmlPath}: ok, nok", ok, nok)
    return desPath


# google image search for exoplanets and upload it html
imPaths = step1_crawler(inferencePool, rglob('/content/*.html'), dirop('/content/dbImgs/unLabeledPlanets', rm=' '))

len(imUrls) 95
/content/exoplanets wallpaper - Google Search.html: ok, nok 71 12


##### step2: Train FastGAN to learn planets distribution and Wait till GAN attains John nash equilibrium

In [ ]:
!rm -rf /content/db
downloadDB('gdrive+https://drive.google.com/file/d/1RHTmh0dWM0Mg-S8_maKv5Up3m6wuwY08/view?usp=sharing', '/content/db')
imPaths = '/content/dbImgs/unLabeledPlanets'
saveDir = f'/content/db/results/{getTimeStamp()}'
ckptPath = rglob('/content/db/**/*.pth')[0]
Path('/content/FastGAN-pytorch/train.py').write_text(trainCode)
!nvidia-smi
!cd /content/FastGAN-pytorch;python train.py --path {imPaths} --ckpt {ckptPath} --im_size 512 --saveDir {saveDir}

____________________________________________________________________________________________________________________

             cd "/content/db";gdown https://drive.google.com/uc?id=1RHTmh0dWM0Mg-S8_maKv5Up3m6wuwY08;

____________________________________________________________________________________________________________________
unzip dataset
zip:  /content/db/good_art_1k_512.zip
returnData:  /content/db
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

Setting up Perceptual loss...
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100% 528M/528M [00:04<00:00, 121MB/s]
Loading model from: /content/FastGAN-pytorch/lpips/weights/v0.1/vgg.pth
...[net-lin [vgg]] initialized
...Done
Namespace(batch_size=6, ckpt='/content/db/good_art_1k_512/models/all_50000.pth', cuda=0, im_size=512, iter=50000, name='test1', path='

##### step3: Generate New Planets

In [ ]:
ckptPath = max(rglob('/content/db/results/**/all_*.pth'), key=lambda x: int(filename(x).split('_')[1]))
print(ckptPath)
desDir = f'/content/db/infer/{getTimeStamp()}'
Path('/content/FastGAN-pytorch/eval.py').write_text(evalCode)
!cd /content/FastGAN-pytorch;python eval.py --n_sample 100 --ckpt {ckptPath} --im_size 512 --desDir {desDir} --seed 4212

/content/db/results/Mar14_01_31_38406482/test1/models/all_0.pth
load checkpoint success, epoch 0
100% 6/6 [01:26<00:00, 14.48s/it]
